
# Exercício:

## Verificação de um programa While

Pretende-se com este exercício verificar a terminação de um programa muito simples pelo recurso ao SMT Z3. Procede-se à definição do sistema de transição de primira ordem (FOTS) respectivo à sua semântica operacional, seguida da verificação de condições relativas à sua terminação.

O programa em questão: assumindo um $a>0$:


1: begin

$\hspace{1cm} x \leftarrow a$

2: while $a>0$ do

$\hspace{1cm} x \leftarrow x-1$

3: exit



In [2]:
import sys, os
from z3 import *

A primeira etapa a considerar é a definição das variáveis para representar espaço de estados do FOTS inerente a este programa. Facilmente se verifica que qualquer momento de execução deste programa fica completamente caracterizado por um par $(x,l)\in \mathbb{Z} \times \{1,2,3\} $, onde $x$ o valor da variável nesse momento, digamos $var$; e $l$ a linha do código onde a execução se encontra, digamos o $place$.

In [3]:
Labels, (var, place) = EnumSort('Labels',['var', 'place'])
Estado = ArraySort(Labels, IntSort())
s = Const('s', Estado)
w = Const('w', Estado)

Teremos agora que introduzir alguns invariantes estruturais do sistema.
O primeiro, "a componente $place$ de um estado $s$, terá que estar numa das 3 linhas do código":

In [4]:
def inv(s):
  l = s[place]
  return(l in [1,2,3])



Inicialização do modelo: a execução terá que começar na primeira linha do código:

In [5]:
def init(s):
  l = s[place]
  return( l==1 )

Agora os invariantes para as transições. Facilmente se extrai do código as três possíveis (classes) de transições:
$trans(x,l,x',l')=\begin{cases}
(l=1) \wedge (l'=2) \wedge (x'>0)\wedge (x'=x)\\
(l=2) \wedge (l'=2) \wedge (x'\geq0) \wedge (x'> x)\\
(l=2) \wedge (l'=3) \wedge (x'=0) \wedge (x'=x)
\end{cases}$

In [6]:
def trans(u,v):
  x = u[var] ; l = u[place] ; x_ = v[var] ; l_ = v[place]
  return Or(
              And( l==1, l_==2, x_>0, x_ == x),
              And( l==2, l_==2, x_>=0, x_>x),
              And( l==2, l_==3, x_==0, x_==x)
            )

Prove-se, por agora, que a relação de transição $trans$ não viola o invariante $inv$. Para tal, pelo princípio de indução é suficiente provar que:
1. $init(s)\rightarrow inv(s)$
2. $trans(l,x,l'x') \wedge inv(l)\rightarrow inv(l')$

são tautologias.

De forma equivalente, e mais adequado ao SMT, podemos provar que as suas formas negadas:
1. $init(s) \wedge \neg inv(s)$ $\hspace{4cm}$(non rooted)
2. $trans(l,x,l'x') \wedge inv(l)\wedge \neg inv(l')$ $\hspace{1cm}$(non_safe)

são insatisfazíveis (i.e. *unsat*).



Templates genéricos para expressar a universalidade de uma
propriedade $P$ (na sua forma negada):

In [7]:

def non_rooted(P):
  return And(init(s), Not(P(s)))


def non_safe(P):
  return And( trans(s,w), P(s), Not(P(w)) )

def non_universal(P):
  return Or( non_rooted(P), non_safe(P))



Template genérico para testar propriedade $P$:

In [8]:
def test(P):
  solver = Solver()
  solver.add(P)
  try:
    print(solver.check())
    print(solver.model())
  except Exception:
    pass

Usando os templates anteriores estamos em condições de provar a pretendida universalidade do invariante $inv$

In [9]:
test(non_universal(inv))

sat
[s = [place -> 1, else -> 1], k!0 = [place -> 1, else -> 1]]


## Análise de terminação 

Partamos agora para a análise da terminação do algoritmo. Para tal, relembremos o resultado das aulas teóricas:

${\textbf Teorema:}$
Dados um FOTS $\Sigma$, uma função de estado $F(s)$ e um predicado $exit(s)$ que caracterize os estados terminais, se forem tautologias 

$(F(s)>0 \vee exit(s)\;,  \; trans(s,s')\rightarrow (F(s)>F(s') \vee exit(s)))$

então verifica-se que $\forall s\in \Sigma.\exists n. exit(s_n)$





Definição do predicado exit:

In [10]:
def nexit(u):
      return (u[var] > 0)

Definição da função de estado:

In [11]:
def f(u):
      return u[var]

Caso base:

In [12]:
def not_anim0(F):
    return And(F(s) <= 0, nexit(s))

In [13]:
test(not_anim0(f))

unsat


Passo indutivo:

In [16]:
def not_anim1(F):
    return And(trans(s,w), F(s) <= F(w), nexit(s))

In [17]:
test(not_anim1(f)) 

sat
[w = [place -> 2, var -> 40, else -> 2],
 s = [place -> 2, var -> 39, else -> 2],
 k!1 = [place -> 2, var -> 39, else -> 2],
 k!2 = [place -> 2, var -> 40, else -> 2]]


## Questão: o que pode concluir este resultado?